In [74]:
import numpy as np
import itertools as it
import functools as ft
import copy

In [69]:
def parse_input(fl) -> list[np.array]:
    grids = []
    with open(fl) as infile:
        chunk = []
        for ln in infile.readlines():
            ln = ln.strip()
            if not ln:
                grids.append(np.array(chunk))
                chunk = []
            else:
                chunk.append(list(ln))
        grids.append(np.array(chunk))
    return grids

In [70]:
test = parse_input("data/day13-test.txt")
inputs = parse_input("data/day13-input.txt")
test

[array([['#', '.', '#', '#', '.', '.', '#', '#', '.'],
        ['.', '.', '#', '.', '#', '#', '.', '#', '.'],
        ['#', '#', '.', '.', '.', '.', '.', '.', '#'],
        ['#', '#', '.', '.', '.', '.', '.', '.', '#'],
        ['.', '.', '#', '.', '#', '#', '.', '#', '.'],
        ['.', '.', '#', '#', '.', '.', '#', '#', '.'],
        ['#', '.', '#', '.', '#', '#', '.', '#', '.']], dtype='<U1'),
 array([['#', '.', '.', '.', '#', '#', '.', '.', '#'],
        ['#', '.', '.', '.', '.', '#', '.', '.', '#'],
        ['.', '.', '#', '#', '.', '.', '#', '#', '#'],
        ['#', '#', '#', '#', '#', '.', '#', '#', '.'],
        ['#', '#', '#', '#', '#', '.', '#', '#', '.'],
        ['.', '.', '#', '#', '.', '.', '#', '#', '#'],
        ['#', '.', '.', '.', '.', '#', '.', '.', '#']], dtype='<U1')]

###### Test

i i+1 e=2i+2

```
0      0         1          (0:i+1,  i+1:2i+2) => (0:1, 1:2)
1      0 1       2 3                           => (0:2, 2:4)
2      0 1 2     3 4 5                         => (0:3, 3:6)
3      0 1 2 3   4 5 6 7                       => (0:4, 4:8)

```

In [71]:
def solve(grids, modify=False):
    tot = 0
    for g in grids:
        vert_col = num_match_cols(g, modify=modify)
        if vert_col is not None:
            tot += vert_col
        else:
            tot += 100*num_match_cols(g.transpose(), modify=modify)
    return tot

def num_match_cols(grid, modify=False):
    grid = copy.deepcopy(grid)
    m, n = grid.shape
    for i in range(n-1):
        # compare the first i columns (0-ixed) with the `i+1:(i+1)+(i+1)`
        # columns reversed
        lt_st = max(0, 2*i+2 - n)
        lt, rt = grid[:,lt_st:i+1], grid[:,i+1:2*i+2][:,::-1]
        if not modify:
            iter_lt_rt = [(lt, rt)]
        else:
            iter_lt_rt = _gen_mutants(lt, rt)
        for _lt, _rt in iter_lt_rt:
            if np.array_equal(_lt,_rt):
                return i+1
    else:
        return None


def _gen_mutants(lt, rt):
    m, n = lt.shape
    for cur_side, other_side in [(lt, rt), (rt, lt)]:
        for i, j in it.product(range(m), range(n)):
            cur = cur_side[i, j]
            cur_side[i, j] = '.' if cur == '#' else '#'
            yield (cur_side, other_side)
            cur_side[i, j] = cur

In [72]:
solve(test), solve(inputs)

(405, 27300)

In [73]:
solve(test, modify=True), solve(inputs, modify=True)

(400, 29276)